In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, GPT2LMHeadModel, GPT2Config
from peft import LoraConfig, PeftModel, get_peft_model
from datasets import load_dataset
from trl import SFTTrainer
from accelerate import Accelerator
from ollama_interact import *

In [2]:
data_name = "vicgalle/alpaca-gpt4"

dataset = load_dataset(
    data_name, 
)

In [3]:
len(dataset['train'])

52002

In [4]:
sentence = dataset['train'][0]

In [5]:
print(sentence)

{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for sta

In [6]:
prompt_prefix = 'תרגם את המשפט הבא מאנגלית לעברית, המשפט באנגלית הוא:'
prompt_softfix = 'והתרגום בעברית הוא:'

In [7]:
def output_handler_pass(m):
    pass

In [ ]:
import requests
file_path = "alpaca_gpt4_hebrew.txt"

with open(file_path, 'a') as file:
    for i in range(len(dataset['train'])):
        instruction = dataset['train'][i]['instruction']
        input = dataset['train'][i]['input']
        output = dataset['train'][i]['output']
        try:
            file.write(f"Q {i}:\n")
            instruction_hebrew = interact_with_ollama(
                prompt = f'{prompt_prefix} {instruction} {prompt_softfix}',
                model = 'DictaLM-2.0-Instruct',
                stream = False,
                output_handler = output_handler_pass
            )
            file.write(f"instruction_hebrew: {instruction_hebrew}\n")
            if input:
                input_hebrew = interact_with_ollama(
                    prompt = f'{prompt_prefix} {input} {prompt_softfix}',
                    model = 'DictaLM-2.0-Instruct',
                    stream = False,
                    output_handler = output_handler_pass
                )
            else:
                input_hebrew = ''
            file.write(f"input_hebrew: {input_hebrew}\n")
            output_hebrew = interact_with_ollama(
                prompt = f'{prompt_prefix} {output} {prompt_softfix}',
                model = 'DictaLM-2.0-Instruct',
                stream = False,
                output_handler = output_handler_pass
            )
            file.write(f"output_hebrew: {output_hebrew}\n")
            file.flush()
        except requests.RequestException as e:
            print(f"An error occurred during the API call: {e}")
        except IOError as e:
            print(f"An error occurred while writing to the file: {e}")